In [1]:
"""purpose - create a persistent DuckDB database and store data from a couple datasets (i.e., as if they had been imported from CSV flatfiles) in it, each as their own table"""
from contextlib import contextmanager
import json
import pathlib as pl
import sys
from typing import Optional, List, Dict, Any, cast
import warnings

import appdirs
import duckdb
import pandas as pd
from pydataset import data
from IPython.display import Markdown


# HACK the path for allow this notebook to import server's modules
THIS_DIR = globals()['_dh'][0]
REPO_ROOT = THIS_DIR.parent
SRC_DIR = (REPO_ROOT/'src').resolve()
#print(f"{THIS_DIR=}\n{REPO_ROOT=}\n{SRC_DIR=}")
sys.path.append(str(SRC_DIR))

from backend.restricted import safe_exec
from backend.sql import base
from backend.sql.duck import DuckDBMonitorMiddleware
from backend.sql.postgres import PostgresMonitorMiddleware
from pages.utils.etc import load_config
config = load_config()


@contextmanager
def ignore_warnings():
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        yield

In [2]:
base.init(wipe=True)
display(base.map_tables_to_connections)

DuckDBMonitorMiddleware.ask_available_tables(conn=<duckdb.duckdb.DuckDBPyConnection object at 0x0000017E2BFC9CB0>)
update_connection_map closing its own connection


{'iris': <backend.sql.base.DuckDbConnectionDetail at 0x17e4e71ee40>,
 'economics': <backend.sql.base.DuckDbConnectionDetail at 0x17e4e731590>,
 'octoprint': <backend.sql.base.DuckDbConnectionDetail at 0x17e4e731090>,
 'pg_foobar': PostgresConnectionDetail(cluster='localhost', cluster_friendly_name='DevBox', port=5432, username='postgres', password='OllyOllyOxenFree', database='foobar'),
 'mfgwh_mx908_exit_tests': PostgresConnectionDetail(cluster='localhost', cluster_friendly_name='DevBox', port=5432, username='postgres', password='OllyOllyOxenFree', database='mfgwh')}

In [3]:
base.map_tables_to_connections['mfgwh_mx908_exit_tests']

PostgresConnectionDetail(cluster='localhost', cluster_friendly_name='DevBox', port=5432, username='postgres', password='OllyOllyOxenFree', database='mfgwh')

In [4]:
df = PostgresMonitorMiddleware.get_dataframe('mfgwh_mx908_exit_tests')
if df is not None:
    # get etc for this table to see if has any postprocessing code
    if etc := next(filter(lambda tup: tup[0] == 'mfgwh_mx908_exit_tests', PostgresMonitorMiddleware.ask_available_tables(with_etc=True)), None):
        etc = cast(Dict[str, Any], etc[1])
        if post_process_code := etc.get('post_process', None):
            df = safe_exec(post_process_code, df=df)['df']

    display(df)


DuckDBMonitorMiddleware.ask_available_tables(conn=<duckdb.duckdb.DuckDBPyConnection object at 0x0000017E4E7C53B0>)
PostgresMonitorMiddleware.get_dataframe(table_name='mfgwh_mx908_exit_tests', conn=<duckdb.duckdb.DuckDBPyConnection object at 0x0000017E4E7C53B0>) -> etc={'cluster': 'localhost', 'cluster_friendly_name': 'DevBox', 'database': 'mfgwh', 'sql': 'SELECT\n   rep.id "report_id",\n   rep.created_at,\n   att.name "attribute",\n   att.dtype,\n   att.is_metadata,\n   CASE \n       WHEN att.dtype = \'STRING\' THEN val.value_string\n       WHEN att.dtype = \'INTEGER\' THEN val.value_integer::text\n       WHEN att.dtype = \'FLOAT\' THEN val.value_float::text\n       WHEN att.dtype = \'BOOLEAN\' THEN val.value_boolean::text\n       WHEN att.dtype = \'DATETIME\' THEN val.value_datetime::text\n       ELSE NULL\n   END "value",\n   COALESCE(dt_val.min_value::text, float_val.min_value::text, int_val.min_value::text) "lsl",\n   COALESCE(dt_val.max_value::text, float_val.max_value::text, int_

value                    \
attribute                            AIR TP Outcome AIR TP Outcome(2)   
report_id created_at                                                    
1         2017-10-03 10:15:00.313329              1                 1   
2         2017-10-03 10:19:07.320474              1                 1   
3         2017-10-03 10:33:50.540050              1                 1   
4         2017-10-04 14:42:40.127446              1                 1   
5         2017-10-04 16:23:11.090822              1                 1   
...                                             ...               ...   
9089      2025-08-15 10:23:22.133890            NaN               NaN   
9090      2025-08-15 10:45:26.026437            NaN               NaN   
9091      2025-08-16 15:04:26.099027            NaN               NaN   
9092      2025-08-16 16:06:39.697077            NaN               NaN   
9093      2025-08-16 16:18:13.962403            NaN               NaN   

                                                                          \
attribute                            AIR TP Outcome(3) AIR TP Outcome(4)   
report_id created_at                                                       
1         2017-10-03 10:15:00.313329                 1                 1   
2         2017-10-03 10:19:07.320474                 1                 1   
3         2017-10-03 10:33:50.540050                 1                 1   
4         2017-10-04 14:42:40.127446                 1                 1   
5         2017-10-04 16:23:11.090822                 1                 1   
...                                                ...               ...   
9089      2025-08-15 10:23:22.133890               NaN               NaN   
9090      2025-08-15 10:45:26.026437               NaN               NaN   
9091      2025-08-16 15:04:26.099027               NaN               NaN   
9092      2025-08-16 16:06:39.697077               NaN               NaN   
9093      2025-08-16 16:18:13.962403               NaN               NaN   

                                                                       \
attribute                            Air TP Outcome Air TP Outcome(2)   
report_id created_at                                                    
1         2017-10-03 10:15:00.313329            NaN               NaN   
2         2017-10-03 10:19:07.320474            NaN               NaN   
3         2017-10-03 10:33:50.540050            NaN               NaN   
4         2017-10-04 14:42:40.127446            NaN               NaN   
5         2017-10-04 16:23:11.090822            NaN               NaN   
...                                             ...               ...   
9089      2025-08-15 10:23:22.133890              1               NaN   
9090      2025-08-15 10:45:26.026437              1               NaN   
9091      2025-08-16 15:04:26.099027              1               NaN   
9092      2025-08-16 16:06:39.697077              1               NaN   
9093      2025-08-16 16:18:13.962403              1               NaN   

                                                                         \
attribute                            Air TP Outcome(3) Blank TP Outcome   
report_id created_at                                                      
1         2017-10-03 10:15:00.313329               NaN              NaN   
2         2017-10-03 10:19:07.320474               NaN              NaN   
3         2017-10-03 10:33:50.540050               NaN              NaN   
4         2017-10-04 14:42:40.127446               NaN              NaN   
5         2017-10-04 16:23:11.090822               NaN              NaN   
...                                                ...              ...   
9089      2025-08-15 10:23:22.133890               NaN                1   
9090      2025-08-15 10:45:26.026437               NaN                1   
9091      2025-08-16 15:04:26.099027               NaN              NaN   
9092      2025-08-16 16:06:39.69707

In [5]:
with ignore_warnings(), duckdb.connect(base.DUCKDB.PATH) as conn:
    usage = pd.read_sql("SELECT * FROM administrative.table_catalog;", conn)
display(usage)

,table_name,table_description,table_type,owned_by,created,updates,updated,hits,last_hit,is_pseudo_table,external_type,etc
0,iris,"A 1936 dataset prepared by statistician Ronald Fisher, contains multivariate data desc...",system,unknown,2025-08-27 12:45:52.584,0,2025-08-27 12:45:52.584,0,2025-08-27 12:45:52.584,False,None,{}
1,economics,This dataset was produced from US economic time series data available from the Federal...,system,unknown,2025-08-27 12:45:52.589,0,2025-08-27 12:45:52.589,0,2025-08-27 12:45:52.589,False,None,{}
2,octoprint,A sensor stream from my personal Octoprint 3D printer server via HomeAssistant...,system,jbechen,2025-08-27 12:45:52.601,0,2025-08-27 12:45:52.601,0,2025-08-27 12:45:52.601,False,None,{}
3,pg_foobar,None,system,unknown,2025-08-27 12:45:52.643,0,2025-08-27 12:45:52.643,0,2025-08-27 12:45:52.643,True,postgres,"{""cluster"":""localhost"",""cluster_friendly_name"":""DevBox"",""database"":""foobar"",""sql"":""SEL..."
4,mfgwh_mx908_exit_tests,None,system,unknown,2025-08-27 12:45:52.651,0,2025-08-27 12:45:52.651,1,2025-08-27 12:45:56.729,True,postgres,"{""cluster"":""localhost"",""cluster_friendly_name"":""DevBox"",""database"":""mfgwh"",""sql"":""SELE..."
